In [ ]:
# Fixed Income Securities

In [ ]:
import numpy as np
from scipy.optimize import fmin

In [ ]:
def price_MBS(r_bar, ka, sigma):
    r0 = 0.078
    WAC = 0.08
    r = WAC/12
    R = 12 * r
    # maturity for MBS
    T = 30
    N = 10_000
    PV_0 = 100_000
    M = T * 12 * 100
    T_CB = np.arange(0,30+1/12,1/12)
    dt = T_CB[-1] / N
    r_1 = CIR(r0, sigma, ka, r_bar, 0, T_CB[-1], N, M)
    positions = (T_CB / T_CB[-1] * M).astype(int)
    Sim_R_dis = np.zeros((N,len(T_CB)))
    for i in range(len(positions)):
        Sim_R_dis[:,i] = -r_1[:,:(positions[i]+1)].mean(axis = 1) * T_CB[i]

    # discount rate for each cash payment 
    discount = np.exp(Sim_R_dis)

    # maturity for 10-year risk free
    T_sim = T + 10
    M_sim = T_sim * 12 * 100

    # approximately for 30-year
    T_CB_sim = np.arange(0,30+1/12,1/12)
    dt = T_sim / M_sim
    r_sim = np.zeros((N,M_sim+1))
    r_sim[:,0] = r0
    for i in range(M):
        dw = np.random.randn(N)
        r_sim[:,i+1] = r_sim[:,i] + ka * (r_bar - r_sim[:,i]) * dt + sigma * np.sqrt(r_sim[:,i]) * np.sqrt(dt) * dw
    positions = (T_CB_sim / T_CB_sim[-1] * M).astype(int)
    r_ten = np.zeros((N,len(T_CB_sim)))
    for i in range(len(positions)):
        # for each time we payment occured corresponds to a rate for CPR
        r_ten[:,i] = r_sim[:,positions[i]:(positions[i]+121)].mean(axis=1)
    
    SY_list = [0.94, 0.76, 0.74, 0.95, 0.98, 0.92, 0.98, 1.10, 1.18, 1.22, 1.23, 0.98]
    PV = np.zeros((N,len(T_CB)))
    PV[:,0] = PV_0
    Ct = np.zeros((N,len(T_CB)))

    # Calculate ct at each time step and recursively calculate the price at 0
    for i in range(1,len(T_CB)):
        # variables to calculate CPR
        RI = 0.28 + 0.14 * np.arctan(-8.57 + 430 * (R - r_ten[:,i-1]))
        BU = 0.3 + 0.7 * (PV[:,i-1]/PV_0)
        SG = np.minimum(1,i/30)
        SY = SY_list[i % 12 - 1]
        CPR = RI * BU * SY * SG
        dis = 1 / (1 - (1 + r) ** ((-T * 12) + (i-1)))
        MP = (PV[:,i-1] * r) * dis
        Ct[:,i] =  MP + (PV[:,i-1] - PV[:,i-1] * r * (dis - 1)) * (1-(1-CPR)**(1/12))
        PV[:,i] = PV[:,i-1] - Ct[:,i] + PV[:,i-1] * r

    # discount cash flow for N paths and then take the mean to get final price
    PV_result = np.sum(Ct[:,1:-1] * discount[:,1:-1].mean(axis = 0)) / N 
    return [PV_result, Ct, Sim_R_dis, N, PV, r]

In [ ]:
PV_result = price_MBS(r_bar = 0.08, ka = 0.6, sigma = 0.12)
print(f'MBS price is: ${PV_result[0]:.2f}')

MBS price is: $100478.28


In [ ]:
r_bar_list = np.arange(0.04, 0.11, 0.01)[:-1]
r_bar_price = []
for i in r_bar_list:
    r_bar_price.append(price_MBS(r_bar = i, ka = 0.6, sigma = 0.12)[0]) 

In [ ]:
r_bar_price

[111313.6880768421,
 109291.02590843024,
 106867.90274997507,
 104146.94012110503,
 100588.58254588883,
 96657.66570857717,
 92362.5757516282]

In [ ]:
Ct = PV_result[1]
Sim_R_dis = PV_result[2]
N = PV_result[3]

def find(x):
    result = np.sum(Ct[:,1:] * np.exp(Sim_R_dis-x)[:,1:].mean(axis = 0)) / N  - 98000
    return result ** 2 

solution = fmin(find,  [0.03])

print(f'OAS is: {solution[0]:.5f}')

Optimization terminated successfully.
         Current function value: 0.000048
         Iterations: 17
         Function evaluations: 34
OAS is: 0.02503


In [ ]:
discount = np.exp(Sim_R_dis)
PV = PV_result[4]
r = PV_result[5]
IP = PV * r
IO = np.sum(IP[:,1:] * discount[:,1:].mean(axis = 0)) / N 
PO = np.sum((Ct[:,1:] - IP[:,1:]) * discount[:,1:].mean(axis = 0)) / N 
print(f'IOs price is: ${IO:.2f}')
print(f'POs price is: ${PO:.2f}')

IOs price is: $40511.85
POs price is: $59971.97
